### Запуск QtLab

In [8]:
%run D:\qtlab_replacement\init.py
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
import pandas as pd
from save_pkl import *
%matplotlib qt5
from matplotlib import interactive
interactive(True)

Failed loading module Labbrick:  [WinError 126] The specified module could not be found
Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


In [51]:
import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)

<module 'resonator_quality_factor_fit' from 'D:\\qtlab_replacement\\scripts\\resonator_quality_factor_fit.py'>

In [3]:
pna = Agilent_N5242A('pna', address = 'PNA')

# Прицеливание

In [74]:
sample_name = 'MGTU Sapphire res 25'
min_freq = 5.0e9
max_freq = 8.0e9
data_root = get_location()

if not os.path.exists('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)):
    os.mkdir('{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name))

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [90]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(32001)
pna.set_bandwidth(1000)
pna.set_power(-10)

True

In [91]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.diff(phase)/(freqs[1]-freqs[0])
delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

In [92]:
sweep.sweep(pna, ([0], lambda x: 0, ''), filename="S21 overview "+sample_name)

A Jupyter Widget

Started at:  Feb 19 2018 17:40:07

Elapsed time: 0 h 0 m 35.91 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


{'S-parameter': (('Frequency',),
  (array([  5.00000000e+09,   5.00009375e+09,   5.00018750e+09, ...,
            7.99981250e+09,   7.99990625e+09,   8.00000000e+09]),),
  array([ 1.38877916-0.24231371j,  1.38071799-0.28477424j,
          1.37142038-0.32803091j, ..., -0.38379997-0.33870921j,
         -0.39302158-0.3265146j , -0.40250316-0.31446323j]),
  {'log': 20},
  ('Hz',))}

In [93]:
threshold = .02

In [94]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = np.abs(np.diff(data_complex))
data_diff[data_diff<threshold] = 0
plt.plot(freqs[1:], data_diff)
peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[1:][peaks]
for freq in peak_freqs:
    print (freq, ',')

found 50 peaks
5052500000.0 ,
5084843750.0 ,
5168750000.0 ,
5174187500.0 ,
5290156250.0 ,
5290437500.0 ,
5411750000.0 ,
5508781250.0 ,
5608906250.0 ,
5704625000.0 ,
5805031250.0 ,
5895031250.0 ,
5895125000.0 ,
5998343750.0 ,
5998437500.0 ,
6092187500.0 ,
6092281250.0 ,
6194375000.0 ,
6194468750.0 ,
6298156250.0 ,
6298343750.0 ,
6394812500.0 ,
6394906250.0 ,
6491843750.0 ,
6491937500.0 ,
6587843750.0 ,
6692656250.0 ,
6692750000.0 ,
6790812500.0 ,
6884468750.0 ,
6884562500.0 ,
6971656250.0 ,
6971750000.0 ,
7076187500.0 ,
7175562500.0 ,
7175656250.0 ,
7271843750.0 ,
7361093750.0 ,
7468250000.0 ,
7559000000.0 ,
7559093750.0 ,
7662593750.0 ,
7802656250.0 ,
7802843750.0 ,
7879250000.0 ,
7879343750.0 ,
7968500000.0 ,
7968593750.0 ,
7968687500.0 ,
7968875000.0 ,


# Мерить зонаторы

In [27]:
resonator_freqs = [ 
5411750000.0 ,
5508781250.0 ,
5608906250.0 ,
5704625000.0 ,
5805031250.0 ,
5895031250.0 ,
5998343750.0 ,
6092187500.0 ,
6194468750.0 ,
6298343750.0 ,
6394812500.0 ,
6491937500.0 ,
6587843750.0 ,
6692750000.0 ,
6790812500.0 ,
6884468750.0 ,
6971656250.0 ,
7076187500.0 ,
7175562500.0 ,
7271843750.0 ,
7361093750.0 ,
7468250000.0 ,
7559000000.0 ,
7662593750.0]
powers = np.linspace(0, -67, 68) # длинный скан
#powers = np.linspace(0, -30, 31) # короткий скан

In [99]:
resonator_ids = [i for i in range(24)]

In [28]:
len(resonator_freqs)

24

In [107]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='notch_port')
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  Feb 19 2018 17:54:51

Elapsed time: 0 h 3 m 31.25 s


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 200.
  warnings.warn(errors[info][0], RuntimeWarning)


A Jupyter Widget

Started at:  Feb 19 2018 17:58:27

Elapsed time: 0 h 3 m 31.2 s


...

Started at:  Feb 19 2018 19:09:56

Elapsed time: 0 h 3 m 29.73 s


A Jupyter Widget

Started at:  Feb 19 2018 19:13:29

Elapsed time: 0 h 3 m 29.51 s


A Jupyter Widget

Started at:  Feb 19 2018 19:17:01

Elapsed time: 0 h 3 m 29.72 s


# load last measurements and refit

In [17]:
measurement[1]

':'

In [75]:
import glob
measurement_pattern = 'D:\\qtlab\\ReiData\\data\\2018-02-19\\*MGTU Sapphire res 25 res {}\\*.pkl'
for resonator_id in range(24):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        #print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            powers = measurement[1][0]
            fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='notch_port')
            fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
            plt.close()
            plt.close()

C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\utilities.py:9: RuntimeWarning: invalid value encountered in log10
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circuit.py:228: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools-0.1-py3.6.egg\resonator_tools\circlefit.py:78: RuntimeWarning: invalid value encountered in minimum
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 800.
  warnings.warn(errors[info][0], RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod

In [ ]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='notch_port')
    fitdata['single_photon_limit'] =  port1.get_single_photon_limit()
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

# Загрузить фиты и построить графики

In [76]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 12
columns = 2
figs, axes = plt.subplots(rows,columns, figsize=(20, 40))
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
resonator_type='notch_port'
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        
        Q_ax = axes[(resonator_id)%rows, (resonator_id)//rows]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = Q_ax.annotate('$f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}, $Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])), \
                          (0.05, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e4, 1e6])
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

    Unnamed: 0    Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  270350.996117  388738.656746    111290.480959  437397.536725   
1            1  269030.747383  388835.733083    102312.857378  436102.102775   
2            2  265847.316840  384283.996380     72139.570613  428518.613615   
3            3  263840.592589  377751.790681     40381.768738  419348.726787   
4            4  263584.326939  371371.218447     29113.422280  410153.539829   
5            5  261722.554855  362391.164429     10469.695551  398073.469382   
6            6  260966.549067  350789.558035      7516.982490  383429.049660   
7            7  260530.927514  340890.985724      3461.002928  371218.566362   
8            8  260463.085760  328152.730835      2764.856802  355157.604143   
9            9  259578.302437  316964.815801      2528.039760  341953.968584   
10          10  258848.551088  299313.112248      1683.422765  320917.306871   
11          11  259246.330502  283729.21

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  3.403985e+05  3.027813e+05     3.982214e+04  3.036311e+05   
1            1  3.357623e+05  3.018804e+05     6.599867e+04  3.026746e+05   
2            2  3.413395e+05  3.065694e+05     4.964015e+04  3.071063e+05   
3            3  3.981217e+05  3.493361e+05     1.092216e+05  3.493486e+05   
4            4  3.329886e+05  2.933294e+05     4.206218e+04  2.936829e+05   
5            5  3.320081e+05  2.901848e+05     1.711460e+04  2.904931e+05   
6            6  3.308670e+05  2.808327e+05     9.422376e+03  2.809835e+05   
7            7  3.962091e+05  3.235144e+05     6.253403e+04  3.236887e+05   
8            8  3.275095e+05  2.638418e+05     3.708640e+03  2.639508e+05   
9            9  3.251285e+05  2.556016e+05     1.920463e+03  2.557200e+05   
10          10  3.277255e+05  2.456719e+05     1.777557e+03  2.457245e+05   
11          11  3.258134e+05  2.288182e+05     1.277516e+03  2.288571e+05   

    Unnamed: 0    Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  248798.686503  147412.743270     15430.269895  169985.473150   
1            1  248497.693958  147612.285171     19755.348481  172060.789002   
2            2  248165.488811  148109.741605     15501.663840  173625.049894   
3            3  248210.320425  146751.016542     12108.933726  173236.959108   
4            4  248070.619603  145241.025027      4585.814326  171811.306972   
5            5  248435.831129  143923.774284      6148.407789  171119.563589   
6            6  248135.200665  141300.454988      2214.274259  168152.144137   
7            7  248951.228073  139296.563690      1365.855603  165599.191468   
8            8  250132.674152  136291.711436       637.440618  161806.289614   
9            9  248023.530331  132534.164605       446.507407  156388.122437   
10          10  249938.475733  128997.927467       444.656412  151973.511064   
11          11  251097.690683  124960.20

[68 rows x 18 columns] 9
    Unnamed: 0   Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  59396.111421  261918.183621      3436.898798  262007.764262   
1            1  58947.532979  257254.470213       904.651696  257352.740305   
2            2  58926.766553  251572.682676      1113.738389  251632.482280   
3            3  58808.462493  245323.921759       863.990124  245375.115946   
4            4  58699.534501  238351.912605       577.441868  238382.476282   
5            5  58711.086379  231187.333871       410.116355  231202.215404   
6            6  58555.236302  223451.058713       417.520794  223465.404015   
7            7  58609.808401  215996.075763       427.550933  216003.209691   
8            8  58471.370609  208516.710989       458.909435  208524.417368   
9            9  58688.757602  201333.156041       434.532670  201336.262779   
10          10  58589.958952  193664.932773       424.222860  193670.963895   
11          11  58578.68658

    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  2.042856e+05  1.897468e+05      4435.624359  1.906323e+05   
1            1  2.070430e+05  1.898313e+05      2584.070867  1.908615e+05   
2            2  2.060399e+05  1.858490e+05      2148.036487  1.869293e+05   
3            3  2.064116e+05  1.823496e+05      1301.967132  1.835144e+05   
4            4  2.086085e+05  1.799681e+05       781.260723  1.811515e+05   
5            5  2.057503e+05  1.734655e+05       697.977943  1.746189e+05   
6            6  2.070433e+05  1.694602e+05       706.963133  1.706598e+05   
7            7  2.069646e+05  1.649863e+05       507.811002  1.661521e+05   
8            8  2.048934e+05  1.587927e+05       592.241135  1.598065e+05   
9            9  2.077124e+05  1.551169e+05       460.459411  1.562155e+05   
10          10  2.069512e+05  1.492197e+05       462.466237  1.502419e+05   
11          11  2.075157e+05  1.439571e+05       420.455757  1.449686e+05   

    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  9.370856e+04  1.822461e+05     2.475311e+03  1.830615e+05   
1            1  9.377913e+04  1.811789e+05     2.089995e+03  1.819424e+05   
2            2  9.453752e+04  1.757729e+05     1.179572e+03  1.766349e+05   
3            3  9.490781e+04  1.706785e+05     1.040905e+03  1.715312e+05   
4            4  9.355181e+04  1.635799e+05     1.999092e+03  1.641795e+05   
5            5  9.518935e+04  1.591569e+05     8.168492e+02  1.599415e+05   
6            6  9.519029e+04  1.512427e+05     7.239917e+02  1.519394e+05   
7            7  9.509440e+04  1.463940e+05     7.064539e+02  1.470146e+05   
8            8  9.575818e+04  1.425393e+05     4.749127e+02  1.432088e+05   
9            9  9.531537e+04  1.369610e+05     7.036317e+02  1.374709e+05   
10          10  9.508138e+04  1.302555e+05     6.696513e+02  1.307114e+05   
11          11  9.609377e+04  1.258846e+05     4.542053e+02  1.263593e+05   

    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  2.379240e+05  2.494295e+05     7.895274e+03  2.678515e+05   
1            1  2.390914e+05  2.505973e+05     1.502242e+04  2.685749e+05   
2            2  2.372480e+05  2.461762e+05     4.839708e+03  2.628734e+05   
3            3  2.358615e+05  2.418087e+05     4.458252e+03  2.576587e+05   
4            4  2.349144e+05  2.371277e+05     2.504456e+03  2.519968e+05   
5            5  2.338010e+05  2.302751e+05     1.144139e+03  2.440968e+05   
6            6  2.355794e+05  2.258974e+05     2.135967e+03  2.384625e+05   
7            7  2.343012e+05  2.190327e+05     1.384240e+03  2.308517e+05   
8            8  2.304707e+05  2.095179e+05     8.366559e+02  2.207158e+05   
9            9  2.341797e+05  2.049665e+05     1.321242e+03  2.148812e+05   
10          10  2.316433e+05  1.964914e+05     8.931556e+02  2.056216e+05   
11          11  2.331308e+05  1.889344e+05     9.488341e+02  1.972904e+05   

    Unnamed: 0    Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  557512.868420  150294.738490       606.905277  156804.335137   
1            1  557950.305090  149078.915334       737.076586  155485.632849   
2            2  559044.000795  143862.607472       527.418357  149858.567278   
3            3  557791.333172  138780.772037       605.105780  144264.848437   
4            4  558861.976779  134536.663357       803.587562  139561.945893   
5            5  556940.665825  129343.096371       723.267850  133920.594502   
6            6  566122.140899  124644.520448      1094.805042  129232.207991   
7            7  553179.141502  122004.421482      1471.261446  125353.396614   
8            8  560891.769311  119732.950079      1257.899144  122937.382654   
9            9  563152.465674  113308.360548      1526.318902  116582.774273   
10          10  549506.625189  108436.438145       882.336976  111345.200392   
11          11  550946.520048  106849.16

In [78]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0     88912
1     86737
2     67690
3     75804
4     76766
5     70556
6     48617
7     53069
8     36750
9     24874
10    31147
11    33566
12    27369
13    72075
14    25725
15    77900
16    37681
17    47956
18    62747
19    80936
20    54084
21    56429
22     8429
Name: Ql_lp, dtype: int32

In [79]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,Qe,Qi_hp,Qi_lp,Ql_hp,Ql_lp,fr,single_photon_limits,k
0,269731.318279,388738.656746,115631.358109,159456.126528,88912.285697,5.692377e+09,-154.453923,21103.879293
1,339969.028189,420879.707044,105819.168594,188737.010441,86736.758292,5.801470e+09,-154.730027,17064.701437
2,200201.756915,347295.123312,90651.902712,127199.604885,67689.860103,5.902237e+09,-153.469897,29481.446529
3,457301.001096,302781.321422,92481.453827,160244.927163,75803.574278,6.002529e+09,-153.035384,13125.990296
4,233119.447301,318540.813969,99559.922172,131852.909449,76765.951591,6.114608e+09,-152.979287,26229.508328
5,444415.955726,283448.830382,82511.072962,163763.547560,70556.242405,6.210245e+09,-152.361917,13973.946093
6,248919.297185,147412.743270,60468.888851,92566.983543,48617.429598,6.308563e+09,-149.198303,25343.807188
7,590040.532930,245132.211818,58598.210671,175700.739158,53068.793810,6.424510e+09,-150.638738,10888.251180
8,78909.143205,272204.327372,69003.484245,61108.177275,36749.634389,6.515155e+09,-150.304592,82565.273061
9,38139.877457,265438.836812,68127.195368,33351.374869,24873.539070,6.637037e+09,-148.034765,174018.309409


# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')